In [1]:
import terl
from terl.envs import TradingEnv, make_env
from terl.config import EnvConfigManager

In [2]:
env = make_env('Test 1')

In [5]:
env._dt_index_map.iloc[-100:-50]

,GBPUSD_15,GBPUSD_60,GBPUSD_240
2020-12-11 09:30:00,543934.0,137537.0,36243.0
2020-12-11 09:45:00,543935.0,137537.0,36243.0
2020-12-11 10:00:00,543936.0,137538.0,36243.0
2020-12-11 10:15:00,543937.0,137538.0,36243.0
2020-12-11 10:30:00,543938.0,137538.0,36243.0
2020-12-11 10:45:00,543939.0,137538.0,36243.0
2020-12-11 11:00:00,543940.0,137539.0,36243.0
2020-12-11 11:15:00,543941.0,137539.0,36243.0
2020-12-11 11:30:00,543942.0,137539.0,36243.0
2020-12-11 11:45:00,543943.0,137539.0,36243.0


In [6]:
env._db

{'GBPUSD_15': #        open                high                low                 close               tick_volume
 0        1.5369999408721924  1.5444999933242798  1.5290000438690186  1.5338000059127808  2781
 1        1.5327999591827393  1.5360000133514404  1.5180000066757202  1.5225000381469727  2571
 2        1.5227999687194824  1.5414999723434448  1.5199999809265137  1.538699984550476   2711
 3        1.5364999771118164  1.5460000038146973  1.530900001525879   1.5355000495910645  2921
 4        1.534999966621399   1.5379999876022339  1.5236999988555908  1.5364999771118164  2711
 ...      ...                 ...                 ...                 ...                 ...
 544,029  1.334879994392395   1.3364900350570679  1.3348400592803955  1.3356000185012817  2456
 544,030  1.3356000185012817  1.338320016860962   1.3356000185012817  1.3377900123596191  2525
 544,031  1.3377900123596191  1.339020013809204   1.3360400199890137  1.338230013847351   4181
 544,032  1.338230013847351   1

In [3]:
cm = EnvConfigManager()

In [5]:
config_checker(cm.get_config('Test 1'))